In [9]:
from google.colab import drive
drive.mount('/content/drive')

FOLDERNAME = 'CS685-Project/s2s-decipherment-multilingual/'

import sys
sys.path.append('/content/drive/My Drive/{}'.format(FOLDERNAME))

%load_ext autoreload
%autoreload 2

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [10]:
%cd /content/drive/My\ Drive/$FOLDERNAME/

/content/drive/.shortcut-targets-by-id/1d104twHY-SPOagFGjLaTFEOZGkx2WNJs/CS685-Project/s2s-decipherment-multilingual


In [5]:
%cd /content/drive/My\ Drive/$FOLDERNAME/code/
%cd /content/drive/My\ Drive/$FOLDERNAME/datasets/gigaword_eng_5_d1/data/afp_eng


/content/drive/.shortcut-targets-by-id/1d104twHY-SPOagFGjLaTFEOZGkx2WNJs/CS685-Project/s2s-decipherment-multilingual/datasets/gigaword_eng_5_d1/data/afp_eng


In [8]:
#!gzip -d afp_eng_199405.gz
#!ls
#!pip install BeautifulSoup4
from code import preprocess

ImportError: ignored

In [ ]:
#from bs4 import BeautifulSoup

In [ ]:
#!cat afp_eng_199405.txt

In [ ]:
#soup = BeautifulSoup(open("afp_eng_199405.txt").read())
#print(soup.prettify())
filename = "afp_eng_199405.txt"
preprocess.preprocess_to_file(filename, "english.train")
preprocess.preprocess_to_file("afp_eng_199406", "english.dev")
preprocess.preprocess_to_file("afp_eng_199407", "english.test")
#soup = BeautifulSoup(f,'html.parser')


In [ ]:
#print(soup.p)

##**Everything below this is just for testing**

In [ ]:
'''docs = soup.find_all('P')
print('paras length:',len(paras))
for doc in docs:
    print(doc)
    #print doc.findChild('P')
'''

paras length: 1


In [ ]:
from lxml.html import fromstring
import lxml.html as PARSER

root = PARSER.fromstring(f)
#root = root.getroottree()
para_list=[]
counter=1
for ele in root.getiterator():
    if ele.tag == "p":
      #if counter<2:
        #print(type(str(ele.text_content())))
        #counter=counter+1
      #print (ele.text_content())
      para_list.append(str(ele.text_content()))

In [ ]:
print(para_list[:4])

['\nTributes poured in from around the world Thursday to the late Labour\nParty leader John Smith, who died earlier from a massive heart attack\naged 55.\n', '\nIn Washington, the US State Department issued a statement regretting\n"the untimely death" of the rapier-tongued Scottish barrister and\nparliamentarian.\n', '\n"Mr. Smith, throughout his distinguished career in government and in\nopposition, left a profound impression on the history of his party and\nhis country," State Department spokesman Michael McCurry said.\n', '\n"Secretary (of State Warren) Christopher extends his deepest\ncondolences to Mrs. Smith and to the Smith children."\n']


In [ ]:
import string
#lower-case all strings
para_list = [each_string.lower() for each_string in para_list]

#remove punctuation, newlines and digits. Replace spaces with underscores
for i in range(len(para_list)):
  para_list[i] = "".join([char for char in para_list[i] if char not in string.punctuation])
  para_list[i] = para_list[i].replace("\n"," ")
  para_list[i] = ''.join([i for i in para_list[i] if not i.isdigit()])
  para_list[i] = para_list[i].strip()
  para_list[i] = para_list[i].replace(" ","_")
  para_list[i] = para_list[i].replace("__","_")
  if len(para_list[i]) > 256:
    para_list[i] = para_list[i][:256]

In [ ]:
print(para_list[:3])

['tributes_poured_in_from_around_the_world_thursday_to_the_late_labour_party_leader_john_smith_who_died_earlier_from_a_massive_heart_attack_aged', 'in_washington_the_us_state_department_issued_a_statement_regretting_the_untimely_death_of_the_rapiertongued_scottish_barrister_and_parliamentarian', 'mr_smith_throughout_his_distinguished_career_in_government_and_in_opposition_left_a_profound_impression_on_the_history_of_his_party_and_his_country_state_department_spokesman_michael_mccurry_said']


In [ ]:
#write to a file
temp = filename.split(".")
preproc_filename = "_p.".join(temp)
with open(preproc_filename, "w+") as fileptr:
  for s in para_list:
        temp_s = " ".join(s)
        fileptr.write("%s\t" % temp_s)
        fileptr.write("\n\n")

In [ ]:
#create substitution cipher
import random

alphabet = 'abcdefghijklmnopqrstuvwxyz'
#key = 'nu.t!iyvxqfl,bcjrodhkaew spzgm'
plaintext = "Hey_this_is_really_fun"


def makeKey(alphabet):
   alphabet = list(alphabet)
   random.shuffle(alphabet)
   return ''.join(alphabet)

def encrypt(plaintext, key, alphabet):
    keyMap = dict(zip(alphabet, key))
    keyMap['_'] = '_'
    return ''.join(keyMap.get(c.lower(), c) for c in plaintext)

def decrypt(cipher, key, alphabet):
    keyMap = dict(zip(key, alphabet))
    return ''.join(keyMap.get(c.lower(), c) for c in cipher)

key = makeKey(alphabet)
cipher = encrypt(plaintext, key, alphabet)

print(plaintext)
print(cipher)
print(decrypt(cipher, key, alphabet))

Hey_this_is_really_fun
abf_hayj_yj_nbceef_wdv
hey_this_is_really_fun


In [ ]:
#compute frequency encoding
from collections import Counter
c_string = "".join(cipher.split('_'))
x = Counter(c_string)
l = x.most_common()
print(l)
freq = {}
for i in range(len(l)):
  freq[l[i][0]] = i
print(freq)
i = 0
freq_enc = ''
for c in cipher:
  freq_enc += '_' if c == '_' else str(freq[c])
  freq_enc += " "
  i = i + 1
print(freq_enc)
# ''.join(freq.get(c.lower(), c) for c in cipher)

[('a', 2), ('b', 2), ('f', 2), ('y', 2), ('j', 2), ('e', 2), ('h', 1), ('n', 1), ('c', 1), ('w', 1), ('d', 1), ('v', 1)]
{'a': 0, 'b': 1, 'f': 2, 'y': 3, 'j': 4, 'e': 5, 'h': 6, 'n': 7, 'c': 8, 'w': 9, 'd': 10, 'v': 11}
0 1 2 _ 6 0 3 4 _ 3 4 _ 7 1 8 5 5 2 _ 9 10 11 


In [ ]:
#import nltk
#nltk.download('all')
from nltk.tokenize import word_tokenize

[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/abc.zip.
[nltk_data]    | Downloading package alpino to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/alpino.zip.
[nltk_data]    | Downloading package biocreative_ppi to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping corpora/biocreative_ppi.zip.
[nltk_data]    | Downloading package brown to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/brown.zip.
[nltk_data]    | Downloading package brown_tei to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/brown_tei.zip.
[nltk_data]    | Downloading package cess_cat to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/cess_cat.zip.
[nltk_data]    | Downloading package cess_esp to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/cess_esp.zip.
[nltk_data]    | Downloading package chat80 to /root/nltk_data...
[nltk_data]    |   Unzipp

In [ ]:
#print(type(para_list))
word_list=[]
x=0
for para in para_list:
  if x<2:
    print(word_tokenize(para))
    x=x+1
  word_list.append(word_tokenize(para))
  #print(word_tokenize(para))


['Tributes', 'poured', 'in', 'from', 'around', 'the', 'world', 'Thursday', 'to', 'the', 'late', 'Labour', 'Party', 'leader', 'John', 'Smith', ',', 'who', 'died', 'earlier', 'from', 'a', 'massive', 'heart', 'attack', 'aged', '55', '.']
['In', 'Washington', ',', 'the', 'US', 'State', 'Department', 'issued', 'a', 'statement', 'regretting', "''", 'the', 'untimely', 'death', "''", 'of', 'the', 'rapier-tongued', 'Scottish', 'barrister', 'and', 'parliamentarian', '.']


In [ ]:
#test_string= 'sticks and stones'
#print(word_tokenize(test_string))

['sticks', 'and', 'stones']
